In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# -----------------------------
# Config
# -----------------------------
IMG_SIZE = 128
BATCH_SIZE = 32
EPOCHS = 10
LR = 0.001
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# -----------------------------
# Image Transforms
# -----------------------------
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# -----------------------------
# Dataset & DataLoader
# -----------------------------
train_dataset = datasets.ImageFolder("dataset/train", transform=train_transform)
val_dataset   = datasets.ImageFolder("dataset/val", transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE)

# -----------------------------
# CNN Model
# -----------------------------
class CatDogCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * (IMG_SIZE // 8) * (IMG_SIZE // 8), 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

model = CatDogCNN().to(DEVICE)

# -----------------------------
# Loss & Optimizer
# -----------------------------
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

# -----------------------------
# Training Loop
# -----------------------------
for epoch in range(EPOCHS):
    model.train()
    train_loss = 0

    for images, labels in train_loader:
        images = images.to(DEVICE)
        labels = labels.float().unsqueeze(1).to(DEVICE)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    print(f"Epoch [{epoch+1}/{EPOCHS}] - Loss: {train_loss/len(train_loader):.4f}")

# -----------------------------
# Validation
# -----------------------------
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        outputs = model(images)
        preds = torch.sigmoid(outputs) > 0.5

        correct += (preds.squeeze() == labels).sum().item()
        total += labels.size(0)

print(f"Validation Accuracy: {100 * correct / total:.2f}%")


Epoch [1/10] - Loss: 0.6931
Epoch [2/10] - Loss: 0.6658
Epoch [3/10] - Loss: 0.6158
Epoch [4/10] - Loss: 0.5478
Epoch [5/10] - Loss: 0.5862
Epoch [6/10] - Loss: 0.4785
Epoch [7/10] - Loss: 0.4406
Epoch [8/10] - Loss: 0.4533
Epoch [9/10] - Loss: 0.4112
Epoch [10/10] - Loss: 0.3265
Validation Accuracy: 83.33%


In [12]:
print(train_dataset.class_to_idx)


{'cats': 0, 'dogs': 1}


In [13]:
from PIL import Image

img = Image.open("cat.jpg").convert("RGB")
img = val_transform(img).unsqueeze(0).to(DEVICE)

model.eval()
with torch.no_grad():
    output = model(img)
    prob = torch.sigmoid(output).item()

print("Dog" if prob > 0.5 else "Cat")


Cat
